In [1]:
!pip install git+https://github.com/bigscience-workshop/promptsource.git

/home/ubuntu/mambafs/mamba-hidden-states


In [ ]:
%%bash
python -m venv mamba_env2
source mamba_env2/bin/activate

#need this for lambda stack
for cudnn_so in /usr/lib/python3/dist-packages/tensorflow/libcudnn*; do
  sudo ln -s "$cudnn_so" /usr/lib/x86_64-linux-gnu/
done

pip install packaging wheel torch ipykernel datasets
pip install accelerate -U
#assume your in mamba directory
pip install .

#install bigcode eval harness
cd 3rdparty
git clone https://github.com/bigcode-project/bigcode-evaluation-harness.git
cd ..
pip install -e 3rdparty/bigcode-evaluation-harness

#install lm eleuther ai eval harness
git submodule update --init --recursive
pip install -e 3rdparty/lm-evaluation-harness
pip install git+https://github.com/bigscience-workshop/promptsource.git
#install flash attention for phi model
pip install flash_attn

#install environment as a ipykernel for use in jupyter notebook
python -m ipykernel install --user --name=mamba_env2

ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn.so.8': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn_adv_infer.so.8': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn_adv_train.so.8': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn_cnn_infer.so.8': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn_cnn_train.so.8': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn_ops_infer.so.8': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libcudnn_ops_train.so.8': File exists


  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached wheel-0.42.0-py3-none-any.whl (65 kB)
  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached ipykernel-6.28.0-py3-none-any.whl (114 kB)
  Using cached datasets-2.16.1-py3-none-any.whl (507 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached fsspec-2023.12.2-py3-none-any.whl (168 kB)
  U

# Eval on squad 2

In [2]:
%%bash
source mamba_env/bin/activate
python evals/lm_harness_eval.py --model mamba \
    --model_args pretrained=state-spaces/mamba-130m \
    --tasks squadv2 --device cuda:0 --batch_size 64


2024-01-12:01:47:13,480 INFO     [utils.py:148] Note: NumExpr detected 30 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-12:01:47:13,480 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-12:01:47:14,626 INFO     [config.py:58] PyTorch version 2.1.2 available.
2024-01-12:01:47:28,442 WARNING  [__init__.py:149] /home/ubuntu/mambafs/mamba-hidden-states/3rdparty/lm-evaluation-harness/lm_eval/tasks/benchmarks/t0_eval.yaml: No module named 'promptsource'. Config will not be added to registry.
2024-01-12:01:47:28,497 WARNING  [__init__.py:149] /home/ubuntu/mambafs/mamba-hidden-states/3rdparty/lm-evaluation-harness/lm_eval/tasks/benchmarks/flan/flan_cot.yaml: No module named 'promptsource'. Config will not be added to registry.
2024-01-12:01:47:33,932 INFO     [__main__.py:184] Selected Tasks: ['squadv2']
config.json: 100%|██████████| 199/199 [00:00<00:00, 1.58MB/s]
Traceback (most recent call last):
  File "/home/ubuntu/mambafs/mamba-hidden-state

CalledProcessError: Command 'b'source mamba_env/bin/activate\npython evals/lm_harness_eval.py --model mamba \\\n    --model_args pretrained=state-spaces/mamba-130m \\\n    --tasks squadv2 --device cuda --batch_size 64\n'' returned non-zero exit status 1.

In [ ]:
%%bash
source mamba_env/bin/activate
python evals/lm_harness_eval.py --model hf \
    --model_args pretrained=EleutherAI/pythia-410m,revision=step100000,dtype="float" \
    --tasks squadv2 --device cuda --batch_size 64

In [ ]:
%%bash
source mamba_env/bin/activate
python evals/lm_harness_eval.py --model hf \
    --model_args pretrained=microsoft/phi-2,torch_dtype="auto", trust_remote_code=True\
    --tasks squadv2 --device cuda --batch_size 64

# Example Model Usage

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

False
2.1.2+cu121


/home/ubuntu/mambafs/mamba-hidden-states/mamba_env2/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [1]:
def test_model(model, tokenizer):
    inputs = tokenizer("Hello, I am", return_tensors="pt")
    tokens = model.generate(**inputs)
    print(tokenizer.decode(tokens[0]))

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
torch.set_default_device("cuda")
model_name = "microsoft/phi-2"
phi = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

test_model(phi, tokenizer)

/home/ubuntu/mambafs/mamba-hidden-states/mamba_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: CUDA driver initialization failed, you might not have a CUDA gpu.

In [5]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer
model_name = "EleutherAI/pythia-410m-deduped"
pythia = GPTNeoXForCausalLM.from_pretrained(model_name,
                                            #revision="step3000",
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name,
  #revision="step3000",
)

test_model(pythia, tokenizer)

/home/ubuntu/mambafs/mamba-hidden-states/mamba_env/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
config.json: 100%|██████████| 570/570 [00:00<00:00, 2.23MB/s]
model.safetensors: 100%|██████████| 911M/911M [00:11<00:00, 81.2MB/s] 
tokenizer_config.json: 100%|██████████| 396/396 [00:00<00:00, 2.58MB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 25.1MB/s]
special_tokens_map.json: 100%|██████████| 99.0/99.0 [00:00<00:00, 685kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello, I am a newbie to the world of programming. I am trying to create a program


In [ ]:
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

torch.set_default_device("cuda")
model_name = "state-spaces/mamba-130m"
mamba = MambaLMHeadModel.from_pretrained(
    model_name, 
    device="cuda", 
    dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

test_model(mamba, tokenizer)

# Finetune on Squadv2 dataset

In [ ]:
from training.train_mamba_with_pythia import *
def run(args):
        
    model = MambaLMHeadModel.from_pretrained(args.model, dtype=torch.bfloat16, device="cuda")

    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer)
    tokenizer.eos_token = "<|endoftext|>"
    tokenizer.pad_token = tokenizer.eos_token

    data_module = SFTDataModule(
        tokenizer=tokenizer,
        data_path=args.data_path,
    )

    trainer = MambaTrainer(
        model=model,
        train_dataset=data_module.dataset,
        tokenizer=tokenizer,
        args=TrainingArguments(
            learning_rate=args.learning_rate,
            num_train_epochs=args.num_epochs,
            per_device_train_batch_size=args.batch_size,
            gradient_accumulation_steps=args.gradient_accumulation_steps,
            optim=args.optim,
            output_dir=args.output,
            save_total_limit=2,
            logging_steps=50,
            save_steps=500,
        ),
        data_collator=data_module.data_collator,
    )

    trainer.train()
    trainer.save_model(args.output)

parser = argparse.ArgumentParser()
parser.add_argument("--model", type=str, default="state-spaces/mamba-130m")
parser.add_argument("--output", type=str, default="output")
parser.add_argument("--tokenizer", type=str, default="EleutherAI/gpt-neox-20b")
parser.add_argument("--learning_rate", type=float, default=5e-4)
parser.add_argument("--batch_size", type=int, default=4)
parser.add_argument("--gradient_accumulation_steps", type=int, default=1)
parser.add_argument("--optim", type=str, default="adamw_torch")
parser.add_argument("--data_path", type=str, default="squad")
parser.add_argument("--num_epochs", type=int, default=10)
args = parser.parse_args('')

run(args)

# Evaluate on Squadv2

In [ ]:
%%bash
python evals/lm_harness_eval.py --model mamba \
    --model_args pretrained=state-spaces/mamba-130m \
    --tasks squadv2 --device cuda --batch_size 64

# APPENDIX

In [ ]:
import torch
from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXModel
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
import sys
torch.set_default_device("cuda")


# Take in the model you want to train
model_name = "state-spaces/mamba-130m"

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

tokenizer_pythia = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m-deduped")
tokenizer_pythia.eos_token = "<|endoftext|>"
tokenizer_pythia.pad_token = tokenizer_pythia.eos_token

In [ ]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer, GPTNeoXModel

pythia = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-410m-deduped",
  #output_hidden_states=True,
  #revision="step3000",
  #cache_dir="./pythia-70m-deduped/step3000",
).to(torch.device('cuda:0'))


In [ ]:
mamba = MambaLMHeadModel.from_pretrained(
    model_name, 
    device="cuda", 
    dtype=torch.float16)

In [ ]:
# Take the user input from the command line
user_message = "Give me three steps to improve my diet, and include some evidence"#input("\n> ")

# Create a prompt
n_shot_prompting = [
    {
        "question": "What is the capital of France?",
        "answer": "Paris"
    },
    {
        "question": "Who invented the segway?",
        "answer": "Dean Kamen"
    },
    {
        "question": "What is the fastest animal?",
        "answer": "Cheetah"
    }
]

prompt = f"You are a Trivia QA bot.\nAnswer the following question succinctly and accurately."
prompt = f"{prompt}\n\n" + "\n\n".join([f"Q: {p['question']}\nA: {p['answer']}" for p in n_shot_prompting])
prompt = f"{prompt}\n\nQ: {user_message}\nA:"

# Debug print to make sure our prompt looks good
print(prompt)

# Encode the text to token IDs
input_ids = torch.LongTensor([tokenizer.encode(prompt)]).cuda()


In [ ]:
# Generate an output sequence of tokens given the input
# "out" will contain the raw token ids as integers
out = model.generate(
    input_ids=input_ids,
    max_length=256,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
pythia_out = pythia(
    input_ids=input_ids,
    output_hidden_states=True
)

mamba_out = mamba(
    input_ids=input_ids,
    output_hidden_states=True
)

In [ ]:
teacher_loss = (pythia_out.logits.softmax(dim=2)[:,:,:50280] - mamba_out.logits.softmax(dim=2)).norm(dim=2).mean()
teacher_loss

In [ ]:
import torch.nn.functional as F
import math
mu = 0
std = math.sqrt(1.0/mamba_out.hidden_states[0].shape[-1])
size = (1, pythia_out.hidden_states[0].shape[-1], mamba_out.hidden_states[0].shape[-1])
W = torch.normal(0, std, size).to(torch.device('cuda:0'))


F.cosine_similarity(pythia_out.hidden_states[0]@ W,  mamba_out.hidden_states[0], dim=2)


In [ ]:
for k,X in enumerate(pythia_out.hidden_states):
    print(k, X.shape)
for k,X in enumerate(mamba_out.hidden_states):
    print(k, X.shape)

In [ ]:
# you must use the tokenizer to decode them back into strings
decoded = tokenizer.batch_decode(out)[0]
print("="*80)
print(decoded)
# out returns the whole sequence plus the original
cleaned = decoded.replace(prompt, "")

# the model will just keep generating, so only grab the first one
# cleaned = cleaned.split("\n\n")[0]
print(cleaned)

/home/ubuntu/test-fs2/project/mamba-hidden-states/mambaphi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Got 0 examples, preprocess...
Tokenizing dataset...


100%|██████████| 87599/87599 [01:57<00:00, 746.07it/s]


Step,Training Loss
50,0.279300
100,0.242800
150,0.301800
200,0.264300
250,0.227300
300,0.267300
350,0.249800
400,0.214700
450,0.216000
500,0.265800


KeyboardInterrupt: 